# import and read

In [84]:
import os
import json
import pandas as pd
import numpy as np

In [85]:
# Pfad zur Excel-Datei
file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\SP3-1.Dateninfrastrukturen\\Pruefung\\input\\Zensus.xlsx'

In [86]:
# Lese die Excel-Datei ein und speichere die Blätter als einzelne DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)


In [87]:
csv_sheets = {sheet_name: data for sheet_name, data in sheets.items() if sheet_name.startswith('CSV')}


In [88]:
sheet_names = csv_sheets.keys()
print(sheet_names)

dict_keys(['CSV-Klassenstufe', 'CSV-Schulform', 'CSV-Hoechster_Schulabschluss', 'CSV-Hoechster_berufl_Abschluss', 'CSV-Erwerbsstatus', 'CSV-ET_Alter', 'CSV-ET_Hoechst_berufl_Abschl', 'CSV-ET_Stellung_im_Beruf', 'CSV-ET_Beruf_Hauptgr_ISCO08', 'CSV-ET_Wirtschaftszweig'])


# transform

In [89]:
# Speichern der DataFrames in separaten Variablen mit angepassten Namen
created_dfs = []
for sheet_name, data in csv_sheets.items():
    new_sheet_name = sheet_name.replace('-', '_')
    globals()[new_sheet_name] = data
    created_dfs.append(new_sheet_name)

# Anzeigen der neuen Namen der erstellten DataFrames
print("Erstellte DataFrames:")
for df_name in created_dfs:
    print(df_name)

Erstellte DataFrames:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


In [ ]:
# Überprüfung auf fehlende Werte in allen DataFrames
for i, df_name in enumerate(created_dfs):
    df = globals()[df_name]
    print(f"Missing values in DataFrame {i + 1} ({df_name}):")
    print(df.isnull().sum())
   

# EDA

In [90]:
df=pd.DataFrame(CSV_Klassenstufe)

In [91]:
# Optionen einstellen
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# DataFrame anzeigen
print(df.head())

   Berichtszeitpunkt          _RS                    Name                          Regionalebene SCHUELER_KLSS_STP SCHUELER_KLSS_STP__M SCHUELER_KLSS_STP__W SCHUELER_KLSS_STP__1 SCHUELER_KLSS_STP__1_M SCHUELER_KLSS_STP__1_W SCHUELER_KLSS_STP__2 SCHUELER_KLSS_STP__2_M SCHUELER_KLSS_STP__2_W SCHUELER_KLSS_STP__3 SCHUELER_KLSS_STP__3_M SCHUELER_KLSS_STP__3_W
0           20220515            0             Deutschland                                   Bund           8903780              4536040              4367740              3048570                1563180                1485400              4490850                2324270                2166580              1364360                 648590                 715760
1           20220515            1      Schleswig-Holstein                                   Land            320060               162880               157180               109240                  54930                  54310               161100                  83860                 

In [92]:
df['Regionalebene'].value_counts()

Regionalebene
Gemeinde                                 10786
Gemeindeverband                           1207
Stadtkreis/kreisfreie Stadt/Landkreis      400
Regierungsbezirk                            29
Land                                        16
Bund                                         1
Name: count, dtype: int64

In [96]:
print(df['Regionalebene'].unique())

['Bund' 'Land' 'Stadtkreis/kreisfreie Stadt/Landkreis' 'Gemeinde'
 'Gemeindeverband' 'Regierungsbezirk']
